In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from skimage import io

import pandas as pd
import numpy as np
import os
import time
from tempfile import TemporaryDirectory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from tqdm.auto import tqdm

C:\Users\Sai Madhavan G\AppData\Local\Temp\ipykernel_26232\3769867654.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_dir = 'dataset'

In [3]:
class CartPoleDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.df.iloc[idx, 0])
        image = io.imread(img_name)/255
        image = np.moveaxis(image, -1, 0)
        image = torch.tensor(image, dtype=torch.float32)
        labels = self.df.iloc[idx, 1:]
        labels = np.array(labels, dtype=np.float32)
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'label': labels}


        return sample

In [4]:
train_ds = CartPoleDataset('dataset/train.csv', data_dir, torchvision.transforms.Resize(224))
test_ds = CartPoleDataset('dataset/test.csv', data_dir, torchvision.transforms.Resize(224))

In [5]:
dataloaders = {"train":DataLoader(train_ds, batch_size=8), "val":DataLoader(test_ds, batch_size=8)}
dataset_sizes = {"train": len(train_ds), "val": len(test_ds)}

In [6]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_loss = float('inf')

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0

                # Iterate over data.
                for d in tqdm(dataloaders[phase]):
                    inputs = d['image']
                    labels = d['label']
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = loss_fn(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)

                epoch_loss = running_loss / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_loss < best_loss:
                    best_loss = epoch_loss
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Loss: {best_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [7]:
model_ft = torchvision.models.resnet18(weights='IMAGENET1K_V1')

In [8]:
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

In [9]:
model_ft = model_ft.to(device)

In [10]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model_ft.parameters())

In [11]:
model_ft = train_model(model_ft, loss, optimizer,
                       num_epochs=25)

Epoch 0/24
----------


  0%|          | 0/2500 [00:00<?, ?it/s]

KeyboardInterrupt: 